In [1]:
import sys
sys.path.append("d:\\OneDrive - Universiteit Utrecht\\Documents\\000 - Documenti\\PROJECTS\\x-tree-search")

In [2]:
from games.tic_tac_toe import TicTacToe
from games.game import GameAgent, User
from algorithms.minimax import MiniMax
from explainers.minimax_explainer import MiniMaxExplainer

# Define game

In [3]:
game = TicTacToe()

# Define tic-tac-toe scoring function for the minimax

In [4]:
import numpy as np
def scoring_function(node):
    """ Evaluate the Tic Tac Toe board state for the 'X' player's perspective """
    state = node.state
    score = 0
    
    # Possible lines to check (3 rows, 3 columns, 2 diagonals)
    lines = []
    # Rows and columns
    for i in range(3):
        lines.append(state[i, :])  # Row
        lines.append(state[:, i])  # Column
    # Diagonals
    lines.append(np.array([state[i, i] for i in range(3)]))  # Main diagonal
    lines.append(np.array([state[i, 2 - i] for i in range(3)]))  # Anti-diagonal

    for line in lines:
        if np.all(line == "X"):
            score += 100  # 'X' wins
        elif np.all(line == "O"):
            score -= 100  # 'O' wins
        elif np.count_nonzero(line == "X") == 2 and np.count_nonzero(line == "free") == 1:
            score += 10  # 'X' is one move away from winning
        elif np.count_nonzero(line == "O") == 2 and np.count_nonzero(line == "free") == 1:
            score -= 10  # 'O' is one move away from winning

    return score

# Define explainer

In [5]:
explainer = MiniMaxExplainer()

In [6]:
explainer.available_frameworks()

lowlevel
highlevel


# Play and Explain

In [7]:
opponent = GameAgent(MiniMax(scoring_function, max_depth=6, use_alpha_beta=False), agent_id=0)
user = User(agent_id=1, ask_what = False)

In [8]:
game.set_players([opponent, user])

In [9]:
await game.start_on_jupyter()

[['O' ' ' ' ']
 ['O' 'X' ' ']
 ['X' ' ' ' ']]

Game paused.

[['O' ' ' ' ']
 ['O' 'X' ' ']
 ['X' ' ' ' ']]


In [10]:
choice = opponent.core.last_choice
explainer.explain_adjective(choice, 'the best', explanation_depth=3, with_framework='highlevel')

{X in (2, 0), id=0_3} is the best (because
 	{X in (2, 0), id=0_3} has possible alternative moves {X in (0, 1), id=0_0}, {X in (0, 2), id=0_1}, {X in (1, 2), id=0_2} (only showing relevant 3)
	and {X in (2, 0), id=0_3} is better than {X in (0, 1), id=0_0} (because
	 	it leads to a better position (because
	 		{X in (2, 0), id=0_3} is not final move
			and {X in (2, 0), id=0_3} has next possible move {O in (0, 2), id=0_3_1}
			and {X in (0, 1), id=0_0} is not final move
			and {X in (0, 1), id=0_0} has next possible move {O in (2, 0), id=0_0_2}))
	and {X in (2, 0), id=0_3} is better than {X in (0, 2), id=0_1} (because
	 	it leads to a better position (because
	 		{X in (2, 0), id=0_3} is not final move
			and {X in (2, 0), id=0_3} has next possible move {O in (0, 2), id=0_3_1}
			and {X in (0, 2), id=0_1} is not final move
			and {X in (0, 2), id=0_1} has next possible move {O in (2, 0), id=0_1_2}))
	and {X in (2, 0), id=0_3} is better than {X in (1, 2), id=0_2} (because
	 	it leads to 

In [11]:
explainer.explain_adjective(choice, 'best', explanation_depth=4, with_framework='lowlevel')

{X in (2, 0), id=0_3} is best ←
 	(assumption) By definition a node is "best" if it's "better" than all "siblings"
	∧ {X in (2, 0), id=0_3} has siblings = {X in (0, 1), id=0_0}, {X in (0, 2), id=0_1}, {X in (1, 2), id=0_2}, {X in (2, 1), id=0_4}, {X in (2, 2), id=0_5} ←
	 		(assumption) Definition of "siblings" is node.parent.children excluding node
	∧ {X in (2, 0), id=0_3} is better than {X in (0, 1), id=0_0} ←
	 		(assumption) By definition, node1 is "better" than node2 if node1 score >= node2 score
			∧ {X in (2, 0), id=0_3} has score = 0 ←
			 			{X in (2, 0), id=0_3} is ¬(leaf) ←
						 				(assumption) Definition of "leaf" is node.is_leaf
						∧ (assumption) Internal nodes have scores from children
						∧ {X in (2, 0), id=0_3} has backpropagating child = {O in (0, 2), id=0_3_1} ←
						 				{X in (2, 0), id=0_3} is opponent player turn
										∧ (assumption) We assume the opponent will do their best move.
										∧ {O in (0, 2), id=0_3_1} is worst
						∧ {O in (0, 2), id=0

In [12]:
explainer.del_explanation_tactic('OnlyRelevantComparisons', to_framework='highlevel', to_adjective='the best')

In [13]:
explainer.explain_adjective(choice, 'best', explanation_depth=2)

{X in (2, 0), id=0_3} is best ←
 	(assumption) By definition a node is "best" if it's "better" than all "siblings"
	∧ {X in (2, 0), id=0_3} has siblings = {X in (0, 1), id=0_0}, {X in (0, 2), id=0_1}, {X in (1, 2), id=0_2}, {X in (2, 1), id=0_4}, {X in (2, 2), id=0_5} ←
	 		(assumption) Definition of "siblings" is node.parent.children excluding node
	∧ {X in (2, 0), id=0_3} is better than {X in (0, 1), id=0_0} ←
	 		(assumption) By definition, node1 is "better" than node2 if node1 score >= node2 score
			∧ {X in (2, 0), id=0_3} has score = 0
			∧ {X in (0, 1), id=0_0} has score = -100
	∧ {X in (2, 0), id=0_3} is better than {X in (0, 2), id=0_1} ←
	 		(assumption) By definition, node1 is "better" than node2 if node1 score >= node2 score
			∧ {X in (2, 0), id=0_3} has score = 0
			∧ {X in (0, 2), id=0_1} has score = -100
	∧ {X in (2, 0), id=0_3} is better than {X in (1, 2), id=0_2} ←
	 		(assumption) By definition, node1 is "better" than node2 if node1 score >= node2 score
			∧ {X in (2